In [1]:
a=2
b=3


In [2]:
c = a+b


In [3]:
c


5

In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)


template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(country_a="Mexico", country_b="Thailand")


chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [10]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        ),
    ]
)

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)


chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

In [13]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

In [14]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

In [15]:
chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [40]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-3.5-turbo", 
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

haikus_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class poet. You create beautiful haikus about the topic which is about a given programming language.",
        ),
        ("human", "Write a haiku about {topic}."),
    ]
)

haikus_chain = haikus_prompt | chat

In [41]:
explain_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class poet critic. You share your critic with us and describe the haiku in a way that is easy to understand for a 5 year old.",
        ),
        ("human", "{haiku}"),
    ]
)


explain_chain = explain_prompt | chat 


final_chain = {"haiku": haikus_chain} | explain_chain

final_chain.invoke({"topic": "python"})



Python's gentle flow,
Indentation like a dance,
Code whispers softly.This haiku is like a dance of words, where the lines flow gently like a river. The way the words are spaced out is like a special secret code that whispers softly to us. It's like a magical spell that makes the words come alive.

AIMessageChunk(content="This haiku is like a dance of words, where the lines flow gently like a river. The way the words are spaced out is like a special secret code that whispers softly to us. It's like a magical spell that makes the words come alive.")

/n {topic} haiku: {haiku}
{explanation}
